In [1]:
%load_ext autoreload
%autoreload 2
import os
import glob
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd

from ultralytics import YOLO

### Let's try to detect jungling balls

Download the demo .mp4 video

In [2]:
# Download the video
VIDEO_PATH = 'videos/traffic.mp4'
!if [ ! -f $VIDEO_PATH ]; then mkdir -p data && wget https://storage.googleapis.com/track-reid/data/jungling_guy.mp4 -O $VIDEO_PATH; fi

In [3]:
from IPython.display import Video
Video(VIDEO_PATH, width=800)

# Video to frames

In [4]:
# You can run this only once:
# Transform this VIDEO_PATH into a list of frames in this folder under frames/
!mkdir -p frames && ffmpeg -i $VIDEO_PATH -vf fps=12 frames/%d.png -hide_banner -loglevel panic

zsh:1: command not found: ffmpeg


In [5]:
available_frames = glob.glob("frames/*.png")
available_frames = sorted(available_frames, key=lambda x: int(x.split("/")[-1].split(".")[0]))

# Detection

In [6]:
%matplotlib inline

MODEL_WEIGHTS = "yolov8m.pt"

model = YOLO(MODEL_WEIGHTS)
results = model(available_frames[0])[0]

plt.imshow(cv2.cvtColor(results.plot(), cv2.COLOR_BGR2RGB))
plt.show()

100%|██████████| 49.7M/49.7M [00:01<00:00, 38.3MB/s]


IndexError: list index out of range

In [ ]:
### We will track only sports ball so we need to filter them out
SPORTS_BALL_CLASS_ID = 32

In [ ]:
from bytetracker import BYTETracker
from bytetracker.basetrack import BaseTrack
from utils import draw_all_bbox_on_image, yolo_results_to_bytetrack_format, scale_bbox_as_xyxy

from trackreid import ReidProcessor

## Tracking without trackreid

In [ ]:
tracker = BYTETracker(track_thresh= 0.15, track_buffer = 3, match_thresh = 0.85, frame_rate= 12)
BaseTrack._count = 0
model = YOLO(MODEL_WEIGHTS, task="detect")

In [ ]:
all_tracked_objects  = []
for frame_id, image_filename in enumerate(available_frames):
    img = cv2.imread(image_filename)
    detections = model.predict(img, classes=[SPORTS_BALL_CLASS_ID], conf=0.15, verbose=False)[0]
    detections_bytetrack_format = yolo_results_to_bytetrack_format(detections)
    tracked_objects = tracker.update(detections_bytetrack_format, frame_id)
    if len(tracked_objects) > 0:
        tracked_objects = np.insert(tracked_objects, 0, frame_id, axis=1)
        all_tracked_objects.append(tracked_objects)

#### • Create a dataframe with the tracking informations

#### • Rescale the bbox to the original image size

In [ ]:
df_tracked = pd.DataFrame(np.concatenate(all_tracked_objects), columns=["frame_id", "x1", "y1", "x2", "y2", "track_id", "class", "confidence"])
df_tracked[["x1", "y1", "x2", "y2"]] = df_tracked[["x1", "y1", "x2", "y2"]].apply(
    lambda x: scale_bbox_as_xyxy(x[0:4], detections.orig_shape), axis=1, result_type="expand"
    )


### Show tracking video

In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'H264') 
OUTPUT_WITH_BBOX_NO_REID = "videos/video_bbox_no_reid.mp4"
out = cv2.VideoWriter(OUTPUT_WITH_BBOX_NO_REID, fourcc, 12, (1280, 720))
for frame_id, image_filename in enumerate(available_frames):
    image = cv2.imread(image_filename)
    if frame_id in df_tracked.frame_id.astype('int').values:
        df_current_frame = df_tracked[df_tracked.frame_id == frame_id][["x1", "y1", "x2", "y2", "track_id", "class", "confidence"]].to_numpy()
        image = draw_all_bbox_on_image(image, df_current_frame)
    out.write(image)
out.release()
print("Video with bounding box without reid is saved at:", OUTPUT_WITH_BBOX_NO_REID)

In [ ]:
print("Number of detected objects: ", len(df_tracked.track_id.unique()))